<a href="https://colab.research.google.com/github/PremKumar-V/ML_Projects/blob/main/Predict_Student_Performance_ML_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Competition - Predict Student Performance from Game Play

## Reference
- [Dataset Link](https://www.kaggle.com/competitions/predict-student-performance-from-game-play/)

## ToDo
- [x] Prepare Notebook
- [ ] Data Exploration
- [ ] Data Preprocessing
- [ ] Model Training
- [ ] Hypertuning
- [ ] Evaluation
- [ ] Submission

### Prepare Notebook

In [5]:
# Download Libraries
!pip install kaggle xgboost opendatasets --quiet

In [6]:
# Import Libraries

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import opendatasets as od
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Download Dataset

od.download('https://www.kaggle.com/competitions/predict-student-performance-from-game-play/data')

100%|██████████| 484M/484M [00:03<00:00, 150MB/s]



Extracting archive ./predict-student-performance-from-game-play/predict-student-performance-from-game-play.zip to ./predict-student-performance-from-game-play


In [8]:
# Inspecting Dataset

dataDir = './predict-student-performance-from-game-play/'

os.listdir(dataDir)

['test.csv',
 'jo_wilder',
 'train.csv',
 'train_labels.csv',
 'sample_submission.csv']

In [9]:
# Load dataset

trainDf = pd.read_csv(dataDir + 'train.csv')
trainDf.head(2)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [10]:
trainLabels = pd.read_csv(dataDir + 'train_labels.csv')

In [11]:
testDf = pd.read_csv(dataDir + 'test.csv')

### Data Exploration

In [12]:
trainDf.describe()

,session_id,index,elapsed_time,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fullscreen,hq,music
count,1.317421e+07,1.317421e+07,1.317421e+07,1.317421e+07,284746.000000,1.213797e+07,1.213797e+07,1.213797e+07,1.213797e+07,1.000737e+06,0.0,0.0,0.0
mean,2.113413e+16,6.526426e+02,3.846817e+06,1.219194e+01,3.157930,-5.493615e+01,-1.162997e+02,4.581144e+02,3.852363e+02,3.186237e+03,NaN,NaN,NaN
std,5.665220e+14,6.275818e+02,2.701387e+07,6.499188e+00,2.064042,5.201468e+02,2.185912e+02,2.472144e+02,1.292879e+02,3.692265e+05,NaN,NaN,NaN
min,2.009031e+16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-1.992355e+03,-9.181587e+02,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN
25%,2.101031e+16,2.890000e+02,4.394300e+05,6.000000e+00,1.000000,-3.529376e+02,-2.128361e+02,2.690000e+02,3.040000e+02,1.000000e+02,NaN,NaN,NaN
50%,2.104022e+16,5.960000e+02,1.013425e+06,1.300000e+01,3.000000,-1.116317e+01,-9.778151e+01,4.470000e+02,3.970000e+02,4.180000e+02,NaN,NaN,NaN
75%,2.110051e+16,8.970000e+02,1.740050e+06,1.800000e+01,5.000000,2.963618e+02,2.268531e+01,6.630000e+02,4.710000e+02,1.266000e+03,NaN,NaN,NaN
max,2.210022e+16,2.047300e+04,1.749293e+09,2.200000e+01,6.000000,1.261774e+03,5.436164e+02,1.916000e+03,1.439000e+03,2.199078e+08,NaN,NaN,NaN


In [13]:
trainDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13174211 entries, 0 to 13174210
Data columns (total 20 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      int64  
 1   index           int64  
 2   elapsed_time    int64  
 3   event_name      object 
 4   name            object 
 5   level           int64  
 6   page            float64
 7   room_coor_x     float64
 8   room_coor_y     float64
 9   screen_coor_x   float64
 10  screen_coor_y   float64
 11  hover_duration  float64
 12  text            object 
 13  fqid            object 
 14  room_fqid       object 
 15  text_fqid       object 
 16  fullscreen      float64
 17  hq              float64
 18  music           float64
 19  level_group     object 
dtypes: float64(9), int64(4), object(7)
memory usage: 2.0+ GB


In [35]:
nullDf = pd.DataFrame((trainDf.isna().sum() / len(trainDf)) * 100)
nullDf

,0
session_id,0.000000
index,0.000000
elapsed_time,0.000000
event_name,0.000000
name,0.000000
level,0.000000
room_coor_x,7.865670
room_coor_y,7.865670
screen_coor_x,7.865670
screen_coor_y,7.865670


In [38]:
validList = list(nullDf[nullDf[0] < 50].T.columns)
validList

['session_id',
 'index',
 'elapsed_time',
 'event_name',
 'name',
 'level',
 'room_coor_x',
 'room_coor_y',
 'screen_coor_x',
 'screen_coor_y',
 'fqid',
 'room_fqid',
 'level_group']

In [29]:
trainDf = trainDf[validList]
len(trainDf)

13174211

In [33]:
originalPercent = 13174210
cutOffPercent = len(trainDf)

print(f"Percentage Cut off: {(originalPercent / cutOffPercent) * 100}")

Percentage Cut off: 99.9999924094126
